In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import warnings
import os

warnings.filterwarnings("ignore")

In [8]:
# ========== CONFIGURATION ==========
DATA_PATH = "../data/engineered_features.csv"
TARGET = "retail_price_£_per_kWh"
MLFLOW_TRACKING_URI = "http://172.210.183.133:8000/"
EXPERIMENT_NAME = "UK Energy - Feature Selection"
TEST_SIZE = 0.2
RANDOM_STATE = 42
OPTUNA_TRIALS = 50

In [5]:
print("UK ENERGY PRICE PREDICTION - FEATURE SELECTION")

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"\nMLflow Tracking URI: {MLFLOW_TRACKING_URI}")
print(f"Experiment: {EXPERIMENT_NAME}")

UK ENERGY PRICE PREDICTION - FEATURE SELECTION


2025/10/23 15:40:57 INFO mlflow.tracking.fluent: Experiment with name 'UK Energy - Feature Selection' does not exist. Creating a new experiment.



MLflow Tracking URI: http://172.210.183.133:8000/
Experiment: UK Energy - Feature Selection


In [9]:
print("\Loading data...")
df = pd.read_csv(DATA_PATH)
print(f"   Loaded data shape: {df.shape}")
print(f"   Date range: {df['datetime'].min()} to {df['datetime'].max()}")


\Loading data...
   Loaded data shape: (1939, 104)
   Date range: 2025-08-02 05:00:00+00:00 to 2025-10-21 23:00:00+00:00


In [10]:
# Exclude non-feature columns
excluded_cols = [TARGET, "datetime"]

# Select only scaled features + temporal features for modeling
features = [col for col in df.columns 
            if col.startswith('scaled_') or 
            col in ['hour_sin', 'hour_cos', 'month_sin', 'month_cos', 
                    'is_weekend', 'is_peak_hour', 'is_night', 
                    'hour', 'day_of_week', 'month']]

# Remove rows with NaN (from lag/rolling features)
df_clean = df[features + [TARGET]].dropna()

X = df_clean[features]
y = df_clean[TARGET]

print(f"   Total features: {len(features)}")
print(f"   Clean samples: {len(X)}")
print(f"   Target: {TARGET}")

   Total features: 50
   Clean samples: 1939
   Target: retail_price_£_per_kWh


In [11]:
print("\nSplitting data...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, shuffle=False  # Time-series: no shuffle
)
print(f"   Train: {len(X_train)} samples")
print(f"   Test: {len(X_test)} samples")


Splitting data...
   Train: 1551 samples
   Test: 388 samples


In [12]:
def calculate_metrics(y_true, y_pred):
    """Calculate regression metrics."""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return {
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "mape": mape
    }

In [13]:
def log_model_to_mlflow(model, model_name, params, metrics, feature_importances):
    """Log model, params, metrics, and artifacts to MLflow."""
    with mlflow.start_run(run_name=model_name):
        # Log parameters
        mlflow.log_params(params)
        
        # Log metrics
        mlflow.log_metrics(metrics)
        
        # Log model
        if "XGBoost" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")
        
        # Log feature importances as artifact
        importance_df = pd.DataFrame({
            'feature': feature_importances.index,
            'importance': feature_importances.values
        })
        importance_path = f"artifacts/{model_name}_feature_importance.csv"
        os.makedirs("artifacts", exist_ok=True)
        importance_df.to_csv(importance_path, index=False)
        mlflow.log_artifact(importance_path)
        
        print(f"   ✅ Logged {model_name} - RMSE: {metrics['rmse']:.4f}, R²: {metrics['r2']:.4f}")


In [14]:
def plot_feature_importance(importances, title, filename):
    """Plot and save feature importance."""
    plt.figure(figsize=(12, 8))
    sns.barplot(x=importances.values, y=importances.index, palette="viridis")
    plt.title(title, fontsize=16, fontweight='bold')
    plt.xlabel("Importance Score", fontsize=12)
    plt.ylabel("Feature", fontsize=12)
    plt.tight_layout()
    
    os.makedirs("plots", exist_ok=True)
    plot_path = f"plots/{filename}"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return plot_path

In [15]:
# ========== RANDOM FOREST - OPTUNA ==========
print("\Training Random Forest with Optuna...")

def rf_objective(trial):
    """Optuna objective for Random Forest."""
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 5, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", 0.5]),
        "random_state": RANDOM_STATE
    }
    
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    
    return mse

\Training Random Forest with Optuna...


In [16]:
rf_study = optuna.create_study(direction="minimize", study_name="RF_Optimization")
rf_study.optimize(rf_objective, n_trials=OPTUNA_TRIALS, show_progress_bar=True)

print(f"\n   🏆 Best RF Parameters:")
for key, value in rf_study.best_params.items():
    print(f"      {key}: {value}")
print(f"   📉 Best MSE: {rf_study.best_value:.6f}")

[I 2025-10-23 15:45:03,354] A new study created in memory with name: RF_Optimization


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-23 15:45:04,979] Trial 0 finished with value: 0.0015211419333231566 and parameters: {'n_estimators': 387, 'max_depth': 7, 'min_samples_split': 11, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 0 with value: 0.0015211419333231566.
[I 2025-10-23 15:45:06,714] Trial 1 finished with value: 0.0013630316043520647 and parameters: {'n_estimators': 365, 'max_depth': 25, 'min_samples_split': 5, 'min_samples_leaf': 7, 'max_features': 'log2'}. Best is trial 1 with value: 0.0013630316043520647.
[I 2025-10-23 15:45:09,983] Trial 2 finished with value: 0.00077519664456508 and parameters: {'n_estimators': 216, 'max_depth': 21, 'min_samples_split': 8, 'min_samples_leaf': 8, 'max_features': 0.5}. Best is trial 2 with value: 0.00077519664456508.
[I 2025-10-23 15:45:17,155] Trial 3 finished with value: 0.0006592987362311459 and parameters: {'n_estimators': 371, 'max_depth': 11, 'min_samples_split': 14, 'min_samples_leaf': 3, 'max_features': 0.5}. Best is trial 3 with value: 0.00

In [17]:
# Train best RF model
best_rf = RandomForestRegressor(**rf_study.best_params, random_state=RANDOM_STATE)
best_rf.fit(X_train, y_train)
rf_preds = best_rf.predict(X_test)
rf_metrics = calculate_metrics(y_test, rf_preds)

In [18]:
# Get feature importances
rf_importances = pd.Series(
    best_rf.feature_importances_, 
    index=X_train.columns
).sort_values(ascending=False)

In [19]:
# Log to MLflow
log_model_to_mlflow(best_rf, "RandomForest_Optuna", rf_study.best_params, rf_metrics, rf_importances[:25])

2025/10/23 15:49:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/23 15:50:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


   ✅ Logged RandomForest_Optuna - RMSE: 0.0231, R²: 0.9178
🏃 View run RandomForest_Optuna at: http://172.210.183.133:8000/#/experiments/1/runs/0c5e29ea80b042e1bcf79556251124ce
🧪 View experiment at: http://172.210.183.133:8000/#/experiments/1


In [20]:
# Plot top 25 features
rf_plot_path = plot_feature_importance(
    rf_importances[:25], 
    "Random Forest - Top 25 Feature Importances",
    "rf_feature_importance.png"
)

In [21]:
with mlflow.start_run(run_name="RandomForest_Optuna"):
    mlflow.log_artifact(rf_plot_path)

print(f"\n   📊 Random Forest Metrics:")
print(f"      RMSE: {rf_metrics['rmse']:.4f}")
print(f"      MAE: {rf_metrics['mae']:.4f}")
print(f"      R²: {rf_metrics['r2']:.4f}")
print(f"      MAPE: {rf_metrics['mape']:.2f}%")

🏃 View run RandomForest_Optuna at: http://172.210.183.133:8000/#/experiments/1/runs/45930e8baae24975ac84706821fd22c1
🧪 View experiment at: http://172.210.183.133:8000/#/experiments/1

   📊 Random Forest Metrics:
      RMSE: 0.0231
      MAE: 0.0147
      R²: 0.9178
      MAPE: 7.03%


In [23]:
# ========== XGBOOST - OPTUNA ==========
print("Training XGBoost with Optuna...")

def xgb_objective(trial):
    """Optuna objective for XGBoost."""
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 2.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 2.0),
        "random_state": RANDOM_STATE,
        "tree_method": "hist"
    }
    
    model = XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    
    return mse

Training XGBoost with Optuna...


In [24]:
xgb_study = optuna.create_study(direction="minimize", study_name="XGB_Optimization")
xgb_study.optimize(xgb_objective, n_trials=OPTUNA_TRIALS, show_progress_bar=True)

print(f"\n   🏆 Best XGBoost Parameters:")
for key, value in xgb_study.best_params.items():
    print(f"      {key}: {value}")
print(f"   📉 Best MSE: {xgb_study.best_value:.6f}")

[I 2025-10-23 15:53:16,710] A new study created in memory with name: XGB_Optimization


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-23 15:53:17,337] Trial 0 finished with value: 0.0016210899002000831 and parameters: {'n_estimators': 186, 'max_depth': 11, 'learning_rate': 0.07184134466962695, 'subsample': 0.6756463925229624, 'colsample_bytree': 0.7378415627966151, 'gamma': 0.06346876026103287, 'reg_alpha': 1.0646731677803267, 'reg_lambda': 1.8486676715160886}. Best is trial 0 with value: 0.0016210899002000831.
[I 2025-10-23 15:53:18,119] Trial 1 finished with value: 0.0027845763967091096 and parameters: {'n_estimators': 413, 'max_depth': 4, 'learning_rate': 0.20289736973616626, 'subsample': 0.7457640075928731, 'colsample_bytree': 0.7614143065260637, 'gamma': 0.5239944171199769, 'reg_alpha': 1.9384042249716809, 'reg_lambda': 0.9946137223428475}. Best is trial 0 with value: 0.0016210899002000831.
[I 2025-10-23 15:53:18,490] Trial 2 finished with value: 0.002658698165109165 and parameters: {'n_estimators': 269, 'max_depth': 4, 'learning_rate': 0.043267649723952264, 'subsample': 0.8248003196690167, 'colsample

In [25]:
best_xgb = XGBRegressor(**xgb_study.best_params, random_state=RANDOM_STATE, tree_method="hist")
best_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
xgb_preds = best_xgb.predict(X_test)
xgb_metrics = calculate_metrics(y_test, xgb_preds)

In [26]:
xgb_importances = pd.Series(
    best_xgb.feature_importances_, 
    index=X_train.columns
).sort_values(ascending=False)

In [27]:
# Log to MLflow
log_model_to_mlflow(best_xgb, "XGBoost_Optuna", xgb_study.best_params, xgb_metrics, xgb_importances[:25])

2025/10/23 15:57:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/23 15:58:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


   ✅ Logged XGBoost_Optuna - RMSE: 0.0282, R²: 0.8776
🏃 View run XGBoost_Optuna at: http://172.210.183.133:8000/#/experiments/1/runs/c055a48ca20b43d694a8411f55fd5b3f
🧪 View experiment at: http://172.210.183.133:8000/#/experiments/1


In [28]:
# Plot top 25 features
xgb_plot_path = plot_feature_importance(
    xgb_importances[:25], 
    "XGBoost - Top 25 Feature Importances",
    "xgb_feature_importance.png"
)

In [29]:
with mlflow.start_run(run_name="XGBoost_Optuna"):
    mlflow.log_artifact(xgb_plot_path)

print(f"\n   📊 XGBoost Metrics:")
print(f"      RMSE: {xgb_metrics['rmse']:.4f}")
print(f"      MAE: {xgb_metrics['mae']:.4f}")
print(f"      R²: {xgb_metrics['r2']:.4f}")
print(f"      MAPE: {xgb_metrics['mape']:.2f}%")

🏃 View run XGBoost_Optuna at: http://172.210.183.133:8000/#/experiments/1/runs/2be6909a65c648b6b403095096e35af5
🧪 View experiment at: http://172.210.183.133:8000/#/experiments/1

   📊 XGBoost Metrics:
      RMSE: 0.0282
      MAE: 0.0156
      R²: 0.8776
      MAPE: 7.22%


In [30]:
# ========== COMPARISON TABLE ==========
print("\n" + "="*60)
print("📊 TOP 25 FEATURES BY MODEL")
print("="*60)

comparison_df = pd.DataFrame({
    "Feature": rf_importances[:25].index,
    "RF_Importance": rf_importances[:25].values,
    "XGB_Importance": xgb_importances[rf_importances[:25].index].values,
    "Avg_Importance": (rf_importances[:25].values + xgb_importances[rf_importances[:25].index].values) / 2
}).sort_values("Avg_Importance", ascending=False)

print("\n" + comparison_df.to_string(index=False))


📊 TOP 25 FEATURES BY MODEL

                         Feature  RF_Importance  XGB_Importance  Avg_Importance
             scaled_price_lag_1h       0.407682        0.148908        0.278295
            scaled_renewable_pct       0.125191        0.209014        0.167102
            scaled_price_lag_24h       0.181102        0.119475        0.150288
                        hour_sin       0.065083        0.059176        0.062130
                            hour       0.031492        0.073562        0.052527
                        hour_cos       0.019684        0.049611        0.034648
            scaled_carbon_lag_1h       0.003285        0.052488        0.027886
  scaled_carbon_intensity_actual       0.035362        0.018089        0.026725
         scaled_carbon_per_price       0.040411        0.012139        0.026275
         scaled_uk_gen_biomass_%       0.022664        0.015893        0.019278
  scaled_log_solar_radiation_Wm2       0.003867        0.021492        0.012679
           

In [31]:
# Save comparison table
comparison_path = "artifacts/feature_importance_comparison.csv"
comparison_df.to_csv(comparison_path, index=False)
print(f"\n💾 Saved comparison table to: {comparison_path}")


💾 Saved comparison table to: artifacts/feature_importance_comparison.csv


In [32]:
print("\n6️⃣ Creating combined feature importance plot...")

fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# RF plot
sns.barplot(ax=axes[0], x=rf_importances[:25].values, y=rf_importances[:25].index, palette="viridis")
axes[0].set_title("Random Forest - Top 25 Features", fontsize=16, fontweight='bold')
axes[0].set_xlabel("Importance Score", fontsize=12)
axes[0].set_ylabel("Feature", fontsize=12)

# XGBoost plot
sns.barplot(ax=axes[1], x=xgb_importances[:25].values, y=xgb_importances[:25].index, palette="plasma")
axes[1].set_title("XGBoost - Top 25 Features", fontsize=16, fontweight='bold')
axes[1].set_xlabel("Importance Score", fontsize=12)
axes[1].set_ylabel("")

plt.tight_layout()
combined_plot_path = "plots/combined_feature_importance.png"
plt.savefig(combined_plot_path, dpi=300, bbox_inches='tight')
plt.close()


6️⃣ Creating combined feature importance plot...


In [33]:
# ========== MODEL COMPARISON ==========
print("📈 MODEL PERFORMANCE COMPARISON")

comparison_metrics = pd.DataFrame({
    "Model": ["Random Forest", "XGBoost"],
    "RMSE": [rf_metrics['rmse'], xgb_metrics['rmse']],
    "MAE": [rf_metrics['mae'], xgb_metrics['mae']],
    "R²": [rf_metrics['r2'], xgb_metrics['r2']],
    "MAPE (%)": [rf_metrics['mape'], xgb_metrics['mape']]
})

print("\n" + comparison_metrics.to_string(index=False))

# Determine best model
best_model_name = "Random Forest" if rf_metrics['rmse'] < xgb_metrics['rmse'] else "XGBoost"
print(f"\n🏆 Best Model: {best_model_name}")

📈 MODEL PERFORMANCE COMPARISON

        Model     RMSE      MAE       R²  MAPE (%)
Random Forest 0.023066 0.014723 0.917841  7.029968
      XGBoost 0.028155 0.015617 0.877587  7.221771

🏆 Best Model: Random Forest


In [34]:
# ========== SAVE RESULTS ==========
print("\Saving results...")

results = {
    "experiment": EXPERIMENT_NAME,
    "mlflow_uri": MLFLOW_TRACKING_URI,
    "data_path": DATA_PATH,
    "target": TARGET,
    "n_features": len(features),
    "n_samples": len(X),
    "train_size": len(X_train),
    "test_size": len(X_test),
    "optuna_trials": OPTUNA_TRIALS,
    "best_model": best_model_name,
    "rf_metrics": rf_metrics,
    "xgb_metrics": xgb_metrics,
    "rf_best_params": rf_study.best_params,
    "xgb_best_params": xgb_study.best_params
}

import json
results_path = "artifacts/feature_selection_results.json"
with open(results_path, 'w') as f:
    # Convert numpy types to native Python types for JSON serialization
    json_results = {k: (float(v) if isinstance(v, (np.floating, np.integer)) else v) 
                    for k, v in results.items() if k not in ['rf_metrics', 'xgb_metrics', 'rf_best_params', 'xgb_best_params']}
    json_results['rf_metrics'] = {k: float(v) for k, v in rf_metrics.items()}
    json_results['xgb_metrics'] = {k: float(v) for k, v in xgb_metrics.items()}
    json_results['rf_best_params'] = rf_study.best_params
    json_results['xgb_best_params'] = xgb_study.best_params
    json.dump(json_results, f, indent=2)

print(f"   ✅ Saved results to: {results_path}")


\Saving results...
   ✅ Saved results to: artifacts/feature_selection_results.json
